<a href="https://colab.research.google.com/github/heokwon/i-seg-you/blob/main/fast_ai_starter_efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Tue Sep 13 03:18:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download

In [3]:
!pip install rasterio efficientnet_pytorch wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 414 kB/s 
     |████████████████████████████████| 1.8 MB 29.6 MB/s 
     |████████████████████████████████| 181 kB 63.4 MB/s 
     |████████████████████████████████| 158 kB 50.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 157 kB 21.3 MB/s 
     |████████████████████████████████| 157 kB 57.4 MB/s 
     |████████████████████████████████| 157 kB 50.8 MB/s 
     |████████████████████████████████| 157 kB 34.9 MB/s 
     |████████████████████████████████| 157 kB 42.9 MB/s 
     |████████████████████████████████| 157 kB 42.3 MB/s 
     |████████████████████████████████| 157 kB 53.7 MB/s 
     |████████████████████████████████| 156 kB 40.6 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=ce7c3a88c7dbd2cf5e8ac6e268f7e9601fa1

In [4]:
!git clone https://github.com/albumentations-team/albumentations.git

Cloning into 'albumentations'...
remote: Enumerating objects: 5139, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 5139 (delta 110), reused 92 (delta 63), pack-reused 4960
Receiving objects: 100% (5139/5139), 99.73 MiB | 33.86 MiB/s, done.
Resolving deltas: 100% (3544/3544), done.


# Wandb login

In [7]:
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "effib4_LPB_768_r2_augp10_diceth.ipynb"

In [8]:
import wandb
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find effib4_LPB_768_r2_augp10_diceth.ipynb.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Project Name

In [9]:
from fastai.callback.wandb import *

# start logging a wandb run
wandb.init(project='HuBMAP + HPA - Hacking the Human Body => Main_testing(LPB_test)')
wandb.run.name = 'effib4_LPB_768_r2_augp10_diceth'

wandb: Currently logged in as: jcjang. Use `wandb login --relogin` to force relogin


# Load Dataset

In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/3. 공모전/참고/Dataset/Converting/Main_dataset/conv_origin_768_r2_binary_dataset.zip'
fantasy_zip = zipfile.ZipFile(zip_path)
fantasy_zip.extractall('/content')

fantasy_zip.close()

In [14]:
# !gdown 1vuUOo20n4Re0LZzJf-Xa248Cho4bZ0Lv
!unzip /content/LPB_768_r2_dataset.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: lpb_conv_768_r2_dataset/masks/27781_0006.png  
  inflating: lpb_conv_768_r2_dataset/masks/8343_0001.png  
  inflating: lpb_conv_768_r2_dataset/masks/676_0001.png  
  inflating: lpb_conv_768_r2_dataset/masks/7706_0003.png  
  inflating: lpb_conv_768_r2_dataset/masks/21501_0005.png  
  inflating: lpb_conv_768_r2_dataset/masks/5583_0004.png  
  inflating: lpb_conv_768_r2_dataset/masks/28262_0004.png  
  inflating: lpb_conv_768_r2_dataset/masks/24194_0003.png  
  inflating: lpb_conv_768_r2_dataset/masks/12026_0001.png  
  inflating: lpb_conv_768_r2_dataset/masks/11448_0000.png  
  inflating: lpb_conv_768_r2_dataset/masks/26319_0002.png  
  inflating: lpb_conv_768_r2_dataset/masks/1878_0003.png  
  inflating: lpb_conv_768_r2_dataset/masks/8502_0003.png  
  inflating: lpb_conv_768_r2_dataset/masks/21812_0004.png  
  inflating: lpb_conv_768_r2_dataset/masks/8450_0005.png  
  inflating: lpb_conv_768_r2_dataset/masks/21358_0003.png  
  inflating:

In [10]:
# train csv
!gdown 1CYzwew45Tuzs1w0oNbraDT4F8RoMlq1O

Downloading...
From: https://drive.google.com/uc?id=1CYzwew45Tuzs1w0oNbraDT4F8RoMlq1O
To: /content/train.csv
100% 16.4M/16.4M [00:00<00:00, 168MB/s]


In [11]:
# b0 ~ b7
!gdown 1Sn2pPVCUz2dpzHB935ai-CU9UmTZDyxp
!unzip ./encoder.zip -d ./encoder

Downloading...
From: https://drive.google.com/uc?id=1Sn2pPVCUz2dpzHB935ai-CU9UmTZDyxp
To: /content/encoder.zip
100% 720M/720M [00:04<00:00, 150MB/s] 
Archive:  ./encoder.zip
  inflating: ./encoder/efficientnet-b0-08094119.pth  
  inflating: ./encoder/efficientnet-b1-dbc7070a.pth  
  inflating: ./encoder/efficientnet-b2-27687264.pth  
  inflating: ./encoder/efficientnet-b3-c8376fa2.pth  
  inflating: ./encoder/efficientnet-b4-e116e8b3.pth  
  inflating: ./encoder/efficientnet-b5-586e6cc6.pth  
  inflating: ./encoder/efficientnet-b6-c76e70fd.pth  
  inflating: ./encoder/efficientnet-b7-dcc49843.pth  


# Import

In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision.all import *
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import cv2
import gc
import random
from albumentations import *
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from lovasz import lovasz_hinge
import rasterio
from rasterio.windows import Window
from tqdm.notebook import tqdm

from PIL import Image
import tifffile as tiff

import warnings
warnings.filterwarnings("ignore")

# Parameter & Etc..

In [15]:
dataset_name = 'effib4_LPB_768_r2_augp10_diceth'
head_epochs = 5
full_epochs = 40
lr_max = 0.5e-2
lr_slice = slice(2e-4,2e-3)
bs = 4
nfolds = 4
fold = 0
SEED = 2020
TRAIN = '/content/lpb_conv_768_r2_dataset/images'
MASKS = '/content/lpb_conv_768_r2_dataset/masks'
LABELS = './train.csv'
NUM_WORKERS = 4
mean = np.array([0.78054955, 0.7566257, 0.7735184]) 
std = np.array([0.25334089, 0.26656769, 0.26202731])

In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    #the following line gives ~10% speedup
    #but may lead to some stochasticity in the results 
    torch.backends.cudnn.benchmark = True
    
seed_everything(SEED)

# Data preprocessing

In [17]:
# # https://www.kaggle.com/iafoss/256x256-images
# mean = np.array([0.65459856,0.48386562,0.69428385])
# std = np.array([0.15167958,0.23584107,0.13146145])


# https://www.kaggle.com/datasets/thedevastator/hubmap-2022-256x256
def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))



class HuBMAPDataset(Dataset):
    def __init__(self, fold=fold, train=True, tfms=None):
        ids = pd.read_csv(LABELS).id.astype(str).values
        kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
        ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
        self.fnames = [fname for fname in os.listdir(TRAIN) if fname.split('_')[0] in ids]
        self.train = train
        self.tfms = tfms
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fname = self.fnames[idx]
        img = cv2.cvtColor(cv2.imread(os.path.join(TRAIN,fname)), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(os.path.join(MASKS,fname),cv2.IMREAD_GRAYSCALE)
        if self.tfms is not None:
            augmented = self.tfms(image=img,mask=mask)
            img,mask = augmented['image'],augmented['mask']
        return img2tensor((img/255.0 - mean)/std),img2tensor(mask)
    


# version 1 0.65 version
def get_aug(p=1.0):
    return Compose([
        HorizontalFlip(p=0.5),
        VerticalFlip(),
        # GridDropout(ratio=0.4),
        RandomRotate90(p=1),
        #Morphology
        ShiftScaleRotate(shift_limit=0, scale_limit=(-0.2,0.2), rotate_limit=(-30,30), 
                         interpolation=1, border_mode=0, value=(0,0,0), p=0.5),
        GaussNoise(var_limit=(0,50.0), mean=0, p=0.5),
        GaussianBlur(blur_limit=(3,7), p=0.5),
        #Color
        RandomBrightnessContrast(brightness_limit=0.35, contrast_limit=0.5, 
                                 brightness_by_max=True,p=0.5),
        HueSaturationValue(hue_shift_limit=30, sat_shift_limit=30, 
                           val_shift_limit=0, p=0.5),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
    ], p=p)

In [18]:
#example of train images with masks
ds = HuBMAPDataset(tfms=get_aug(p=1.0))
dl = DataLoader(ds,batch_size=8,shuffle=False,num_workers=NUM_WORKERS)
imgs,masks = next(iter(dl))

plt.figure(figsize=(16,16))
for i,(img,mask) in enumerate(zip(imgs,masks)):
    img = ((img.permute(1,2,0)*std + mean)*255.0).numpy().astype(np.uint8)
    plt.subplot(8,8,i+1)
    plt.imshow(img,vmin=0,vmax=255)
    plt.imshow(mask.squeeze().numpy(), alpha=0.2)
    plt.axis('off')
    plt.subplots_adjust(wspace=None, hspace=None)
    
del ds,dl,imgs,masks

# Model - EfficientUnet

In [19]:
class FPN(nn.Module):
    def __init__(self, input_channels:list, output_channels:list):
        super().__init__()
        self.convs = nn.ModuleList(
            [nn.Sequential(nn.Conv2d(in_ch, out_ch*2, kernel_size=3, padding=1),
             nn.ReLU(inplace=True), nn.BatchNorm2d(out_ch*2),
             nn.Conv2d(out_ch*2, out_ch, kernel_size=3, padding=1))
            for in_ch, out_ch in zip(input_channels, output_channels)])
        
    def forward(self, xs:list, last_layer):
        hcs = [F.interpolate(c(x),scale_factor=2**(len(self.convs)-i),mode='bilinear') 
               for i,(c,x) in enumerate(zip(self.convs, xs))]
        hcs.append(last_layer)
        return torch.cat(hcs, dim=1)

class UnetBlock(Module):
    def __init__(self, up_in_c:int, x_in_c:int, nf:int=None, blur:bool=False,
                 self_attention:bool=False, **kwargs):
        super().__init__()
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        self.bn = nn.BatchNorm2d(x_in_c)
        ni = up_in_c//2 + x_in_c
        nf = nf if nf is not None else max(up_in_c//2,32)
        self.conv1 = ConvLayer(ni, nf, norm_type=None, **kwargs)
        self.conv2 = ConvLayer(nf, nf, norm_type=None,
            xtra=SelfAttention(nf) if self_attention else None, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, up_in:Tensor, left_in:Tensor) -> Tensor:
        s = left_in
        up_out = self.shuf(up_in)
        cat_x = self.relu(torch.cat([up_out, self.bn(s)], dim=1))
        return self.conv2(self.conv1(cat_x))
        
class _ASPPModule(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, padding, dilation, groups=1):
        super().__init__()
        self.atrous_conv = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                stride=1, padding=padding, dilation=dilation, bias=False, groups=groups)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_conv(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class ASPP(nn.Module):
    def __init__(self, inplanes=512, mid_c=256, dilations=[6, 12, 18, 24], out_c=None):
        super().__init__()
        self.aspps = [_ASPPModule(inplanes, mid_c, 1, padding=0, dilation=1)] + \
            [_ASPPModule(inplanes, mid_c, 3, padding=d, dilation=d,groups=4) for d in dilations]
        self.aspps = nn.ModuleList(self.aspps)
        self.global_pool = nn.Sequential(nn.AdaptiveMaxPool2d((1, 1)),
                        nn.Conv2d(inplanes, mid_c, 1, stride=1, bias=False),
                        nn.BatchNorm2d(mid_c), nn.ReLU())
        out_c = out_c if out_c is not None else mid_c
        self.out_conv = nn.Sequential(nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False),
                                    nn.BatchNorm2d(out_c), nn.ReLU(inplace=True))
        self.conv1 = nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False)
        self._init_weight()

    def forward(self, x):
        x0 = self.global_pool(x)
        xs = [aspp(x) for aspp in self.aspps]
        x0 = F.interpolate(x0, size=xs[0].size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat([x0] + xs, dim=1)
        return self.out_conv(x)
    
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

In [20]:
pretrained_root = './encoder/'
efficient_net_encoders = {
    "efficientnet-b0": {
        "out_channels": (3, 32, 24, 40, 112, 320),
        "stage_idxs": (3, 5, 9, 16),
        "weight_path": pretrained_root + "efficientnet-b0-08094119.pth"
    },
    "efficientnet-b1": {
        "out_channels": (3, 32, 24, 40, 112, 320),
        "stage_idxs": (5, 8, 16, 23),
        "weight_path": pretrained_root + "efficientnet-b1-dbc7070a.pth"
    },
    "efficientnet-b2": {
        "out_channels": (3, 32, 24, 48, 120, 352),
        "stage_idxs": (5, 8, 16, 23),
        "weight_path": pretrained_root + "efficientnet-b2-27687264.pth"
    },
    "efficientnet-b3": {
        "out_channels": (3, 40, 32, 48, 136, 384),
        "stage_idxs": (5, 8, 18, 26),
        "weight_path": pretrained_root + "efficientnet-b3-c8376fa2.pth"
    },
    "efficientnet-b4": {
        "out_channels": (3, 48, 32, 56, 160, 448),
        "stage_idxs": (6, 10, 22, 32),
        "weight_path": pretrained_root + "efficientnet-b4-e116e8b3.pth"
    },
    "efficientnet-b5": {
        "out_channels": (3, 48, 40, 64, 176, 512),
        "stage_idxs": (8, 13, 27, 39),
        "weight_path": pretrained_root + "efficientnet-b5-586e6cc6.pth"
    },
    "efficientnet-b6": {
        "out_channels": (3, 56, 40, 72, 200, 576),
        "stage_idxs": (9, 15, 31, 45),
        "weight_path": pretrained_root + "efficientnet-b6-c76e70fd.pth"
    },
    "efficientnet-b7": {
        "out_channels": (3, 64, 48, 80, 224, 640),
        "stage_idxs": (11, 18, 38, 55),
        "weight_path": pretrained_root + "efficientnet-b7-dcc49843.pth"
    }
}

In [21]:
import sys
# sys.path.insert(0, '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

from efficientnet_pytorch import EfficientNet
from efficientnet_pytorch.utils import url_map, url_map_advprop, get_model_params

class EfficientNetEncoder(EfficientNet):
    def __init__(self, stage_idxs, out_channels, model_name, depth=5):

        blocks_args, global_params = get_model_params(model_name, override_params=None)
        super().__init__(blocks_args, global_params)
        
        cfg = efficient_net_encoders[model_name]

        self._stage_idxs = stage_idxs
        self._out_channels = out_channels
        self._depth = depth
        self._in_channels = 3

        del self._fc
        self.load_state_dict(torch.load(cfg['weight_path']))

    def get_stages(self):
        return [
            nn.Identity(),
            nn.Sequential(self._conv_stem, self._bn0, self._swish),
            self._blocks[:self._stage_idxs[0]],
            self._blocks[self._stage_idxs[0]:self._stage_idxs[1]],
            self._blocks[self._stage_idxs[1]:self._stage_idxs[2]],
            self._blocks[self._stage_idxs[2]:],
        ]

    def forward(self, x):
        stages = self.get_stages()

        block_number = 0.
        drop_connect_rate = self._global_params.drop_connect_rate

        features = []
        for i in range(self._depth + 1):

            # Identity and Sequential stages
            if i < 2:
                x = stages[i](x)

            # Block stages need drop_connect rate
            else:
                for module in stages[i]:
                    drop_connect = drop_connect_rate * block_number / len(self._blocks)
                    block_number += 1.
                    x = module(x, drop_connect)

            features.append(x)

        return features

    def load_state_dict(self, state_dict, **kwargs):
        state_dict.pop("_fc.bias")
        state_dict.pop("_fc.weight")
        super().load_state_dict(state_dict, **kwargs)  
        

class EffUnet(nn.Module):
    def __init__(self, model_name, stride=1):
        super().__init__()
        
        cfg = efficient_net_encoders[model_name]
        stage_idxs = cfg['stage_idxs']
        out_channels = cfg['out_channels']
        
        self.encoder = EfficientNetEncoder(stage_idxs, out_channels, model_name)

        #aspp with customized dilatations
        self.aspp = ASPP(out_channels[-1], 256, out_c=384, 
                         dilations=[stride*1, stride*2, stride*3, stride*4])
        self.drop_aspp = nn.Dropout2d(0.5)
        #decoder
        self.dec4 = UnetBlock(384, out_channels[-2], 256)
        self.dec3 = UnetBlock(256, out_channels[-3], 128)
        self.dec2 = UnetBlock(128, out_channels[-4], 64)
        self.dec1 = UnetBlock(64, out_channels[-5], 32)
        self.fpn = FPN([384, 256, 128, 64], [16]*4)
        self.drop = nn.Dropout2d(0.1)
        self.final_conv = ConvLayer(32+16*4, 1, ks=1, norm_type=None, act_cls=None)
        
    def forward(self, x):
        enc0, enc1, enc2, enc3, enc4 = self.encoder(x)[-5:]
        enc5 = self.aspp(enc4)
        dec3 = self.dec4(self.drop_aspp(enc5), enc3)
        # dec3 = self.dec4(enc5, enc3)
        dec2 = self.dec3(dec3,enc2)
        dec1 = self.dec2(dec2,enc1)
        dec0 = self.dec1(dec1,enc0)
        x = self.fpn([enc5, dec3, dec2, dec1], dec0)
        x = self.final_conv(self.drop(x))
        # x = self.final_conv(x)
        x = F.interpolate(x,scale_factor=2,mode='bilinear')
        return x
    
#split the model to encoder and decoder for fast.ai
split_layers = lambda m: [
                list(m.encoder.parameters()),
                list(m.aspp.parameters())+list(m.dec4.parameters())+
                list(m.dec3.parameters())+list(m.dec2.parameters())+
                list(m.dec1.parameters())+list(m.fpn.parameters())+
                list(m.final_conv.parameters())
            ]

# Loss & Metrics

In [22]:
def symmetric_lovasz(outputs, targets):
    return 0.5*(lovasz_hinge(outputs, targets) + lovasz_hinge(-outputs, 1.0 - targets))

In [23]:
class Dice_soft(Metric):
    def __init__(self, axis=1): 
        self.axis = axis 
    def reset(self): self.inter,self.union = 0,0
    def accumulate(self, learn):
        pred,targ = flatten_check(torch.sigmoid(learn.pred), learn.y)
        self.inter += (pred*targ).float().sum().item()
        self.union += (pred+targ).float().sum().item()
    @property
    def value(self): return 2.0 * self.inter/self.union if self.union > 0 else None
    
# dice with automatic threshold selection
class Dice_th(Metric):
    def __init__(self, ths=np.arange(0.1,0.9,0.05), axis=1): 
        self.axis = axis
        self.ths = ths
        
    def reset(self): 
        self.inter = torch.zeros(len(self.ths))
        self.union = torch.zeros(len(self.ths))
        
    def accumulate(self, learn):
        pred,targ = flatten_check(torch.sigmoid(learn.pred), learn.y)
        for i,th in enumerate(self.ths):
            p = (pred > th).float()
            self.inter[i] += (p*targ).float().sum().item()
            self.union[i] += (p+targ).float().sum().item()

    @property
    def value(self):
        dices = torch.where(self.union > 0.0, 
                2.0*self.inter/self.union, torch.zeros_like(self.union))
        return dices.max()

# Model evaluation

In [24]:
#iterator like wrapper that returns predicted and gt masks
class Model_pred:
    def __init__(self, model, dl, tta:bool=True, half:bool=False):
        self.model = model
        self.dl = dl
        self.tta = tta
        self.half = half
        
    def __iter__(self):
        self.model.eval()
        name_list = self.dl.dataset.fnames
        count=0
        with torch.no_grad():
            for x,y in iter(self.dl):
                x = x.cuda()
                if self.half: x = x.half()
                p = self.model(x)
                py = torch.sigmoid(p).detach()
                if self.tta:
                    #x,y,xy flips as TTA
                    flips = [[-1],[-2],[-2,-1]]
                    for f in flips:
                        p = self.model(torch.flip(x,f))
                        p = torch.flip(p,f)
                        py += torch.sigmoid(p).detach()
                    py /= (1+len(flips))
                if y is not None and len(y.shape)==4 and py.shape != y.shape:
                    py = F.upsample(py, size=(y.shape[-2],y.shape[-1]), mode="bilinear")
                py = py.permute(0,2,3,1).float().cpu()
                batch_size = len(py)
                for i in range(batch_size):
                    taget = y[i].detach().cpu() if y is not None else None
                    yield py[i],taget,name_list[count]
                    count += 1
                    
    def __len__(self):
        return len(self.dl.dataset)
    
class Dice_th_pred(Metric):
    def __init__(self, ths=np.arange(0.1,0.9,0.01), axis=1): 
        self.axis = axis
        self.ths = ths
        self.reset()
        
    def reset(self): 
        self.inter = torch.zeros(len(self.ths))
        self.union = torch.zeros(len(self.ths))
        
    def accumulate(self,p,t):
        pred,targ = flatten_check(p, t)
        for i,th in enumerate(self.ths):
            p = (pred > th).float()
            self.inter[i] += (p*targ).float().sum().item()
            self.union[i] += (p+targ).float().sum().item()

    @property
    def value(self):
        dices = torch.where(self.union > 0.0, 2.0*self.inter/self.union, 
                            torch.zeros_like(self.union))
        return dices
    
def save_img(data,name,out):
    data = data.float().cpu().numpy()
    img = cv2.imencode('.png',(data*255).astype(np.uint8))[1]
    out.writestr(name, img)

# Train

In [ ]:
# ## callbacks - early stopping, save model
# cbs=[EarlyStoppingCallback(monitor='valid_loss', min_delta=0, patience=5),
#      SaveModelCallback(monitor='dice_th',comp=np.greater)]

In [25]:
dice = Dice_th_pred(np.arange(0.2,0.7,0.01))
for fold in range(1):
    ds_t = HuBMAPDataset(fold=fold, train=True, tfms=get_aug(p=1.0))
    # ds_t = HuBMAPDataset(fold=fold, train=True)
    ds_v = HuBMAPDataset(fold=fold, train=False)
    data = ImageDataLoaders.from_dsets(ds_t,ds_v,bs=bs,
                num_workers=NUM_WORKERS,pin_memory=True).cuda()
    model = EffUnet('efficientnet-b4').cuda()
    learn = Learner(data, model, loss_func=symmetric_lovasz,
                metrics=[Dice_soft(),Dice_th()], 
                splitter=split_layers,
                cbs=WandbCallback())
    # .to_fp16(clip=0.5)
    
    #start with training the head
    learn.freeze_to(-1) #doesn't work
    for param in learn.opt.param_groups[0]['params']:
        param.requires_grad = False
    learn.fit_one_cycle(head_epochs, lr_max=lr_max)

    #continue training full model
    learn.unfreeze()
    learn.fit_one_cycle(full_epochs, lr_max=lr_slice,
        # cbs=SaveModelCallback(monitor='valid_loss',comp=np.less))
        cbs=SaveModelCallback(monitor='dice_th',comp=np.greater))
    torch.save(learn.model.state_dict(),f'model_{fold}.pth')
    
    #model evaluation on val and saving the masks
    mp = Model_pred(learn.model,learn.dls.loaders[1])
    with zipfile.ZipFile('val_masks_tta.zip', 'a') as out:
        for p in progress_bar(mp):
            dice.accumulate(p[0],p[1])
            save_img(p[0],p[2],out)
    gc.collect()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'HuBMAPDataset' object has no attribute 'items'


epoch,train_loss,valid_loss,dice_soft,dice_th,time
0,1.606463,1.592713,0.314838,0.456650,05:49
1,1.572516,1.489423,0.319303,0.519310,05:44
2,1.898825,1.552809,0.377663,0.573290,05:43
3,1.429168,1.417003,0.395614,0.615215,05:45
4,1.433432,1.405228,0.395063,0.629597,05:46


epoch,train_loss,valid_loss,dice_soft,dice_th,time
0,1.423099,1.359788,0.424252,0.655645,07:21
1,1.380058,1.313015,0.457061,0.668120,07:18
2,1.352177,1.264315,0.507331,0.721396,07:21
3,1.267331,1.179607,0.583284,0.755819,07:13
4,1.191380,1.106914,0.655317,0.797426,07:14
5,1.138779,1.034612,0.670447,0.806410,07:14
6,1.087773,0.976735,0.725476,0.821911,07:34
7,1.033358,0.999886,0.720617,0.823991,07:15
8,1.027719,0.977522,0.751036,0.838840,07:14
9,1.006815,0.895484,0.764556,0.845959,07:19


Better model found at epoch 0 with dice_th value: 0.6556446552276611.
Better model found at epoch 1 with dice_th value: 0.6681197285652161.
Better model found at epoch 2 with dice_th value: 0.7213958501815796.
Better model found at epoch 3 with dice_th value: 0.7558192014694214.
Better model found at epoch 4 with dice_th value: 0.7974257469177246.
Better model found at epoch 5 with dice_th value: 0.8064097166061401.
Better model found at epoch 6 with dice_th value: 0.8219113349914551.
Better model found at epoch 7 with dice_th value: 0.8239905834197998.
Better model found at epoch 8 with dice_th value: 0.8388404846191406.
Better model found at epoch 9 with dice_th value: 0.8459587693214417.
Better model found at epoch 11 with dice_th value: 0.8502717018127441.
Better model found at epoch 12 with dice_th value: 0.8534018397331238.
Better model found at epoch 14 with dice_th value: 0.8577548861503601.
Better model found at epoch 15 with dice_th value: 0.8588660955429077.
Better model fou

In [26]:
dices = dice.value
noise_ths = dice.ths
best_dice = dices.max()
best_thr = noise_ths[dices.argmax()]
plt.figure(figsize=(8,4))
plt.plot(noise_ths, dices, color='blue')
plt.vlines(x=best_thr, ymin=dices.min(), ymax=dices.max(), colors='black')
d = dices.max() - dices.min()
plt.text(noise_ths[-1]-0.1, best_dice-0.1*d, f'DICE = {best_dice:.3f}', fontsize=12);
plt.text(noise_ths[-1]-0.1, best_dice-0.2*d, f'TH = {best_thr:.3f}', fontsize=12);
plt.show()

In [27]:
!mkdir {dataset_name}
!mv ./model_0.pth ./{dataset_name}
# !mv ./model_1.pth ./{dataset_name}
# !mv ./model_2.pth ./{dataset_name}
# !mv ./model_3.pth ./{dataset_name}

In [28]:
!zip /content/drive/MyDrive/data/kaggle/main_test/{dataset_name}.zip -r ./{dataset_name}

  adding: effib4_LPB_768_r2_augp10_diceth/ (stored 0%)
  adding: effib4_LPB_768_r2_augp10_diceth/model_0.pth (deflated 7%)
